In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os
from pathlib import Path
import cv2 as cv
from feat import Detector
from feat.utils import FEAT_EMOTION_COLUMNS

In [3]:
path = Path(os.getcwd()).parent
DIR_PATH = str(Path("__file__").parent.parent.absolute()) + r"\\"

In [13]:
df = Path(str(path) + '/data/extracted_df.csv')
data = pd.read_csv(df)

In [14]:
data.columns

Index(['AU01', 'AU02', 'AU04', 'AU05', 'AU06', 'AU07', 'AU09', 'AU10', 'AU11',
       'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU23', 'AU24', 'AU25', 'AU26',
       'AU28', 'AU43', 'anger', 'disgust', 'fear', 'happiness', 'sadness',
       'surprise', 'neutral', 'input', 'valence', 'arousal', 'expression'],
      dtype='object')

In [15]:
data['expression'].value_counts()

expression
0    410
1    336
3    166
6    159
2     89
4     72
5     53
Name: count, dtype: int64

In [30]:
df_to_work = data[['expression', 'AU01', 'AU02', 'AU04', 'AU05', 'AU06', 'AU07', 'AU09', 'AU10', 'AU11', 'AU12', 'AU14', 'AU15', 'AU17', 'AU20', 'AU23', 'AU24', 'AU25', 'AU26', 'AU28', 'AU43']]

In [24]:
expression = {"anger": 0, "disgust": 1, "fear": 2, "happiness": 3, "neutral": 4, "sadness": 5, "surprise": 6}

In [31]:
df_to_work

,expression,AU01,AU02,AU04,AU05,AU06,AU07,AU09,AU10,AU11,...,AU14,AU15,AU17,AU20,AU23,AU24,AU25,AU26,AU28,AU43
0,6,0.483270,0.243675,0.541549,0.338625,0.543061,1.0,0.564238,0.039407,0.0,...,0.397873,0.118658,0.192308,1.0,0.366773,0.022223,0.998066,0.425452,0.057370,0.366805
1,6,0.430284,0.180681,0.265370,0.270021,0.095835,0.0,0.337251,0.000563,1.0,...,0.315263,0.192611,0.470449,0.0,0.261354,0.189194,0.019923,0.107538,0.076298,0.119555
2,6,0.175819,0.165525,0.547125,0.274774,0.663774,1.0,0.603282,0.146711,1.0,...,0.313485,0.291478,0.404496,1.0,0.263181,0.417918,0.992757,0.476209,0.020674,0.561816
3,3,0.510513,0.432733,0.577490,0.438239,0.159838,0.0,0.352225,0.007023,0.0,...,0.226066,0.168697,0.358096,0.0,0.605431,0.187319,0.745002,0.169120,0.326765,0.263470
4,6,0.443179,0.390055,0.764268,0.291717,0.244650,1.0,0.401869,0.013133,0.0,...,0.255710,0.387399,0.402032,1.0,0.532307,0.285596,0.916017,0.138788,0.228614,0.359586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1280,3,0.281361,0.524596,0.388748,0.322978,0.130899,0.0,0.313867,0.471087,0.0,...,0.657972,0.100130,0.241004,1.0,0.543075,0.069487,0.997558,0.496915,0.040536,0.124136
1281,3,0.383748,0.175175,0.359569,0.412944,0.136093,1.0,0.461420,0.575754,1.0,...,0.198488,0.243687,0.354316,1.0,0.331048,0.256194,0.979614,0.151429,0.046544,0.144686
1282,6,0.285846,0.257504,0.408956,0.406746,0.099004,0.0,0.252234,0.303962,0.0,...,0.136741,0.069544,0.340935,1.0,0.669794,0.136436,0.861503,0.092502,0.150691,0.287367
1283,6,0.355674,0.201202,0.738301,0.364928,0.232533,1.0,0.495352,0.010874,0.0,...,0.215868,0.655007,0.376874,1.0,0.770400,0.219904,0.967132,0.271063,0.222076,0.402276


In [26]:
import numpy as np
import pandas as pd
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch

In [43]:
class MLP(nn.Module):
    def __init__(self, features_in=2, features_out=3):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(features_in, 100),
            nn.ReLU(),
            nn.Linear(100, features_out),
        )

    def forward(self, input):
        return self.net(input)

In [44]:
class MultiEmoVA(Dataset):
    def __init__(self, data):
        super().__init__()

        # everything in pytorch needs to be a tensor
        self.inputs = torch.tensor(data.drop("expression", axis=1).to_numpy(dtype=np.float32))

        # we need to transform label (str) to a number. In sklearn, this is done internally
        self.index2label = [label for label in data["expression"].unique()]
        label2index = {label: i for i, label in enumerate(self.index2label)}

        self.labels = torch.tensor(data["expression"].apply(lambda x: torch.tensor(label2index[x])))

    def __getitem__(self, index):
        return self.inputs[index], self.labels[index]

    def __len__(self):
        return len(self.inputs)

In [47]:
from tqdm import tqdm

dataset = MultiEmoVA(df_to_work)
K = 10
# passing a generator to random_split is similar to specifying the seed in sklearn
generator = torch.Generator().manual_seed(2023)
# we need to move our model to the correct device
cross_validation = []
# it is common to do a training loop multiple times, we call these 'epochs'
for k in tqdm(range(K)):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")
    
    train, test = random_split(dataset, [0.8, 0.2], generator=generator)
    
    print("Number of objects in Training set: ", len(train))
    print("Number of objects in Validation set: ", len(test))
    
    train_loader = DataLoader(train, batch_size=11, shuffle=True)
    loss_fn = nn.CrossEntropyLoss()
    model = MLP(train[0][0].shape[0], len(dataset.index2label)).to(device)
    optim = torch.optim.SGD(model.parameters(), lr=0.01)

    max_epochs = 200
    for epoch in tqdm(range(max_epochs)):
        for inputs, labels in train_loader:
            # both input, output and model need to be on the same device
            inputs = inputs.to(device)
            labels = labels.to(device)

            out = model(inputs)
            loss = loss_fn(out, labels)

            loss.backward()
            optim.step()
            optim.zero_grad()
            
    # print(f"Training epoch {epoch} average loss: {loss:.4f}")
    # tell pytorch we're not training anymore
    with torch.no_grad():
        test_loader = DataLoader(test, batch_size=4)
        correct = 0
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            predictions = model(inputs)

            # Here we go from the models output to a single class and compare to ground truth
            correct += (predictions.softmax(dim=1).argmax(dim=1) == labels).sum()
        print(f"Accuracy is: {correct / len(test) * 100:0.4f}%")
    k_run_accuracy = correct / len(test) * 100
    cross_validation.append(k_run_accuracy)
print(f"Mean accuracy: {sum(cross_validation) / len(cross_validation):0.4f}%")

  0%|          | 0/10 [00:00<?, ?it/s]

Using device: cuda
Number of objects in Training set:  1028
Number of objects in Validation set:  257



 10%|█         | 1/10 [00:25<03:53, 25.97s/it]

Accuracy is: 67.7043%
Using device: cuda
Number of objects in Training set:  1028
Number of objects in Validation set:  257



 20%|██        | 2/10 [00:51<03:26, 25.84s/it]

...

Accuracy is: 71.9844%
Using device: cuda
Number of objects in Training set:  1028
Number of objects in Validation set:  257



 90%|█████████ | 9/10 [03:54<00:26, 26.05s/it]

Accuracy is: 67.3152%
Using device: cuda
Number of objects in Training set:  1028
Number of objects in Validation set:  257



 90%|█████████ | 9/10 [04:09<00:27, 27.75s/it]


KeyboardInterrupt: 

In [1]:
print("Hello")

Hello
